In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from surprise.prediction_algorithms.matrix_factorization import SVDpp, SVD
from surprise.prediction_algorithms.knns import KNNBasic

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [31]:
from scipy import sparse
from lightfm import LightFM


In [10]:
users = pd.read_csv('goodreads2/user_w_uid_bavg_titles.csv')
div_m = pd.read_csv('div_array.csv')

In [11]:
users.head()

,book_id,user_id,rating,goodreads_book_id,is_div,uid,bid,b_average_rating,title
0,1,3662,4,2767052,0,152.0,0.0,4.34,The Hunger Games
1,1,5461,3,2767052,0,206.0,0.0,4.34,The Hunger Games
2,1,11927,4,2767052,0,396.0,0.0,4.34,The Hunger Games
3,1,24845,5,2767052,0,785.0,0.0,4.34,The Hunger Games
4,1,15494,5,2767052,0,513.0,0.0,4.34,The Hunger Games


In [6]:
rat_sparse.loc[rat_sparse['4']!=0].head()

,0,1,2,3,4,5,6,7,8,9,...,9491,9492,9493,9494,9495,9496,9497,9498,9499,9500
155,4.0,0.0,4.0,0.0,4.0,5.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
209,3.0,4.0,2.0,0.0,5.0,0.0,5.0,4.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
260,0.0,0.0,4.0,0.0,5.0,0.0,3.0,5.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
359,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
393,0.0,4.0,0.0,5.0,5.0,3.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
rat_imp.head()

,0,1,2,3,4,5,6,7,8,9,...,9491,9492,9493,9494,9495,9496,9497,9498,9499,9500
0,4.34,4.44,3.57,4.25,3.89,4.26,4.25,3.79,3.85,4.24,...,3.68,3.73,4.12,4.32,3.09,4.09,4.25,4.35,3.65,4.0
1,4.34,4.44,3.57,4.25,3.89,4.26,4.25,3.79,3.85,4.24,...,3.68,3.73,4.12,4.32,3.09,4.09,4.25,4.35,3.65,4.0
2,4.34,4.44,3.57,4.25,3.89,5.00,4.25,3.79,3.85,4.24,...,3.68,3.73,4.12,4.32,3.09,4.09,4.25,4.35,3.65,4.0
3,4.34,4.44,3.57,4.25,3.89,4.26,4.25,3.79,3.85,4.24,...,3.68,3.73,4.12,4.32,3.09,4.09,4.25,4.35,3.65,4.0
4,4.34,4.44,3.57,4.25,3.89,4.26,4.25,3.79,3.85,4.24,...,3.68,3.73,4.12,4.32,3.09,4.09,4.25,4.35,3.65,4.0


In [5]:
div_m.head()

,bid,is_div
0,0.0,0
1,1.0,0
2,2.0,0
3,3.0,0
4,4.0,0


In [121]:
#Train test split for LightFM model

def get_train_test(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=5, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test

In [96]:
train, test= train_test_split(rat_sparse.values)

In [97]:
train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [59]:
#initialize LightFM model
lfm = LightFM(no_components=50)

In [60]:
train[train==1] = -1
train[train==2] = -1
train[train>2] = 1

In [98]:
trn = sparse.csr_matrix(train)
tst = sparse.csr_matrix(test)

In [102]:
lfm.fit(trn,epochs=30)

In [103]:
pred_lfm1 = lfm.predict(test[:100,0],test[:100,1])

In [64]:
trn.shape

(1517, 9501)

In [65]:
tst.shape

(1517, 9501)

In [55]:
rat_sparse.head()

,0,1,2,3,4,5,6,7,8,9,...,9491,9492,9493,9494,9495,9496,9497,9498,9499,9500
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
test[:10,:20]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [104]:
pred_lfm1

array([3.73442101, 3.73442101, 3.73442101, 3.73442101, 3.73442101,
       3.73442101, 3.73442101, 3.73442101, 3.73442101, 3.73442101,
       3.73442101, 3.73442101, 3.73442101, 3.73442101, 3.73442101,
       3.73442101, 3.73442101, 3.73442101, 3.73442101, 3.73442101,
       3.73442101, 3.73442101, 3.73442101, 3.73442101, 3.73442101,
       3.73442101, 3.73442101, 3.73442101, 3.73442101, 3.73442101,
       3.73442101, 3.73442101, 3.73442101, 3.73442101, 3.73442101,
       3.73442101, 3.73442101, 3.73442101, 3.73442101, 3.73442101,
       3.73442101, 3.73442101, 3.73442101, 3.73442101, 3.73442101,
       3.73442101, 3.73442101, 3.73442101, 3.73442101, 3.73442101,
       3.73442101, 3.73442101, 3.73442101, 3.73442101, 3.73442101,
       3.73442101, 3.73442101, 3.73442101, 3.73442101, 3.73442101,
       3.73442101, 3.73442101, 3.73442101, 3.73442101, 3.73442101,
       3.73442101, 3.73442101, 3.73442101, 3.73442101, 3.73442101,
       3.73442101, 3.73442101, 3.73442101, 3.73442101, 3.73442

In [12]:
#Copy dataframe to start preparing for surprise

df = users.copy()

In [7]:
#import necessary surprise packages

from surprise import Reader, Dataset
from surprise.model_selection import train_test_split

In [8]:
df.head()

,book_id,user_id,rating,goodreads_book_id,is_div,uid,bid,b_average_rating
0,1,3662,4,2767052,0,152.0,0.0,4.34
1,1,5461,3,2767052,0,206.0,0.0,4.34
2,1,11927,4,2767052,0,396.0,0.0,4.34
3,1,24845,5,2767052,0,785.0,0.0,4.34
4,1,15494,5,2767052,0,513.0,0.0,4.34


In [13]:
#Initialize surprise dataset and split into training and test set

from surprise_functions import load_surprise_data, build_model, titles_from_ids

ratings, train = load_surprise_data(df)



In [14]:
#Train surprise K-nearest neighbor model
surprise_sim_opt = {'name':'cosine','user_based':False}

knn = KNNBasic(k=100, min_k=20,sim_options = surprise_sim_opt)

knn.fit(train)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [27]:
#predict on the test set with k-nearest neighbor model
test = train.build_anti_testset()
#pred_knn = knn.test(test)

In [47]:
len(test)

14321975

In [37]:
from surprise import accuracy

In [217]:
accuracy.mae(pred_knn)

MAE:  0.7568


0.7567747966622804

In [32]:
#Initialize surprise SVD model

svd = SVD(n_epochs = 50)

In [33]:
#fit training set data

svd.fit(train)

In [34]:
#predict on test set data

pred_svd = svd.test(test)

In [35]:
len(pred_svd)

14321975

In [44]:
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns: top_n (array): array of shape (num_users, n) with top n predictions for each user id.
    
    '''

    # Creates a matrix with shape (n_users, n_books).
    rat_pred = np.zeros((1517,9501))
    
    # Fills the matrix with the average rating for book weighted by a factor of 0.8 to ensure booksthat are personally matched with users gets returned first.
    for bid in range(rat_pred.shape[1]):
        rat_pred[:,bid] = users.loc[users['bid']==bid]['b_average_rating'].iloc[0]*0.8*div_m.iloc[int(bid)][1]

    #Fills in actual prediction for given user id, book id combination where the estimate is non-zero.
    for uid, bid, true_r, est, _ in predictions:
        if est > 0.6:
            rat_pred[int(uid)][int(bid)] = est*div_m.iloc[int(bid)][1]

    # Sorts the predictions for each user and retrieve the n highest ones.
    top_n = np.zeros((rat_pred.shape[0],n))
    
    for uid in range(rat_pred.shape[0]):
        top_n[uid] = rat_pred[uid].argsort()[::-1][:n]
            
    return top_n


In [41]:
pred_svd[2]

Prediction(uid=152.0, iid=6.0, r_ui=3.8152301401997217, est=4.191640661411605, details={'was_impossible': False})

In [ ]:
pd.DataFrame(pred_svd, columns =['user_id', 'book_id', 'r_ui', 'rating' ,'details']).to_csv('prediction_surprise_svd.csv',index=None)

In [138]:
def get_n_rec_user(df,user_id,model,testset,n=5):
    
    tst = []
    for (uid,bid,rat) in testset:
        if uid == user_id:
            tst.append((uid,bid,rat))
        continue
        
    pred = model.test(tst)
    
    # Creates a matrix with shape (n_books).
    rat_pred = np.zeros(9501)
    
    # Fills the matrix with the average rating for book weighted by a factor of 0.8 to ensure booksthat are personally matched with users gets returned first.
    for bid in range(len(rat_pred)):
        rat_pred[bid] = users.loc[users['bid']==bid]['b_average_rating'].iloc[0]*0.8*div_m.iloc[int(bid)][1]

    #Fills in actual prediction for given user id, book id combination where the estimate is non-zero.
    for uid, bid, true_r, est, _ in pred:
        if est > 0.6:
            rat_pred[int(bid)] = est*div_m.iloc[int(bid)][1]

    # Sorts the predictions for each user and retrieve the n highest ones.
    top_n = rat_pred.argsort()[::-1][:n]
    rec_n = titles_from_ids(df,bids=list(top_n))
    
    return rec_n

In [141]:
rec_svd_1500 = get_n_rec_user(users,1500,svd,test)

In [142]:
rec_svd_1500

['The Song of Achilles',
 'Rose Under Fire',
 'Into the Wilderness',
 'Chains',
 'Nefertiti']

In [62]:
div_m.iloc[8787].is_div

1.0

In [105]:
titles_from_ids(users,bids=list(rec_svd_1500))

['The Song of Achilles',
 'Rose Under Fire',
 'Into the Wilderness',
 'Chains',
 'Nefertiti']

In [304]:
type(rat_data)

surprise.dataset.DatasetAutoFolds

In [92]:
from surprise import dump

In [112]:
dump.dump('../surprise_svd_dump_file', algo=svd)

In [113]:
_,model = dump.load('../surprise_svd_dump_file')

In [114]:
rec_svd_1500 = get_n_rec_user(1500,model,test,n=5)

In [106]:
type(model)

surprise.prediction_algorithms.matrix_factorization.SVD

In [108]:
def get_Iu(uid):
    """Return the number of items rated by given user
    
    Args:
        uid: The raw id of the user.
    Returns:
        The number of items rated by the user.
    """
    
    try:
        return len(train.ur[train.to_inner_uid(uid)])
    except ValueError:  # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """Return the number of users that have rated given item
    
    Args:
        iid: The raw id of the item.
    Returns:
        The number of users that have rated the item.
    """
    
    try:
        return len(train.ir[train.to_inner_iid(iid)])
    except ValueError:  # item was not part of the trainset
        return 0

In [115]:
test[:5]

[(152.0, 1.0, 3.8152301401997217),
 (152.0, 3.0, 3.8152301401997217),
 (152.0, 6.0, 3.8152301401997217),
 (152.0, 7.0, 3.8152301401997217),
 (152.0, 8.0, 3.8152301401997217)]

In [133]:
df2 = pd.DataFrame(test)

In [137]:
test_df = pd.read_csv('testset.csv')

In [136]:
df2.to_csv('testset.csv')

In [ ]:
test_df